In [1]:
import numpy as np
import pandas as pd
import re
from cleantext import clean
import string
from math import exp
from random import seed
from random import random
from random import randrange
from csv import reader

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
import re 
import pandas as pd
try:
    import cPickle as pickle
except ImportError:
    import pickle  
import numpy as np

with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text
#instead of removing emojis, converts them to text 
df=pd.read_excel('Copy of Bot Data.xlsx')

#convert text to lower case 
df['biography']=df['biography'].str.lower()
df['comment']=df['comment'].str.lower()

#expand contractions 
contractions_dict = {"aren't": "are not", "don't": "do not", "Don't": "do not", "I'm": "I am", "i'm": "I am", 
                    "it's": "it is", "y'all": "you all", "Y'all": "you all", "didn't": "did not", "won't": "will not",
                   "I'll": "I will", "i'll": "I will", "can't": "can not"}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text, expand_dict):
    def replace(match):
        return expand_dict[match.group(0)]
    return contractions_re.sub(replace, text)
df['biography']=df['biography'].apply(lambda x:expand_contractions(x, contractions_dict ))
df['comment']=df['comment'].apply(lambda x:expand_contractions(x, contractions_dict))

bios=df['biography'].astype(str)
usernames=df['Username']
comments=df['comment']
new_bios=[]
for b in bios: 
    temp=convert_emojis_to_word(b)
    new_bios.append(temp)
new_comments=[]
for c in comments:
    temp=convert_emojis_to_word(c)
    new_comments.append(temp)
df_filtered=df.copy()
df_filtered['biography']=new_bios
df_filtered['comment']=new_comments
from cleantext import clean

new_bios=[]
for b in df_filtered['biography']: 
    temp=clean(b, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_bios.append('None')
    else: 
        new_bios.append(temp)
new_comments=[]
for c in df_filtered['comment']:
    temp=clean(c, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_comments.append('None')
    else: 
        new_comments.append(temp)

df_filtered['biography']=new_bios
df_filtered['comment']=new_comments
df_filtered['combined']=df_filtered['biography'] + ' ' + df_filtered['comment']


In [3]:
#classifying the biographies and comments combined 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer1 =  CountVectorizer(min_df=0, lowercase=False)
vectorizer1.fit(df_filtered['combined'])
text_array = vectorizer1.transform(df_filtered['combined']).toarray()
text_df = pd.DataFrame(data=text_array,columns = vectorizer1.get_feature_names())

from sklearn.feature_selection import RFE

model = LogisticRegression(max_iter=1000)
selector1=RFE(estimator=model, n_features_to_select=50, step=1) #chose arbitrarily to select 50 of the most important words out of 2000 to make it more manageable

#split dataset into features and labels
Y1=df_filtered['bot classification (0-not a bot, 1-bot)']
X1=text_df

#run RFE
rfe1 = selector1.fit_transform(X1, Y1)
filter1=(selector1.get_support())

##
#filter columns using data from RFE
filter1=list(filter1)
current_cols = list(X1.columns)

#figure out which columns to keep
important_cols=[]
for index in range(len(filter1)):
    if (filter1[index])==True:
        important_cols.append(current_cols[index])
text_df=text_df[important_cols]
df_combined=pd.concat([text_df, df_filtered], axis=1)
df_combined.replace([np.inf, -np.inf], np.nan, inplace=True)
df_combined=df_combined.dropna()
df_combined=df_combined.drop(['comment', 'biography', 'Username', 'combined'], axis=1)
df_combined=df_combined.sample(frac=1, random_state=10)

In [9]:
df_combined.to_csv('data.csv', index=False)
df_combined

,000,100,app,babedrooling_face,bf,bio,bit,bitcoin,charlotte_empire,christiana_menkova__fx,...,vilbl,we,youtube,Follower Count,Following Count,Follower/Following Ratio,Number of posts,# of likes on comment,time of comment after post (minutes),"bot classification (0-not a bot, 1-bot)"
170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2372,297,7.986532,25,0,12,0
282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,912,2412,0.378109,295,440,3,0
132,1,1,0,0,0,0,0,0,0,0,...,0,0,0,3401,108,31.490741,0,0,2,1
65,0,0,0,0,0,1,0,0,0,0,...,0,0,0,26,268,0.097015,0,96,0,1
24,0,0,1,0,0,0,0,0,0,0,...,0,0,0,7,35,0.200000,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,0,0,1,0,0,0,0,0,0,0,...,0,0,0,34,238,0.142857,0,23,1,1
123,1,0,0,0,0,0,0,0,0,0,...,0,0,0,2532,1504,1.683511,9,0,1,1
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,39,0.102564,0,1,0,1
125,0,0,0,0,0,1,0,0,0,0,...,0,0,0,571,4,142.750000,3,163,1,1


In [10]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [11]:
from math import exp
from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation
 
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - expected[j])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


In [12]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))


In [13]:
dataset = load_csv('data.csv')

for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)

str_column_to_int(dataset, len(dataset[0])-1)
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

train=dataset[:251]
test=dataset[251:]

#put everything together
n_folds = 5
learning_rate = 0.1
n_epoch = 500
n_layers = 5
n_inputs = len(train[0]) - 1
n_outputs = len(set([row[-1] for row in train]))
network = initialize_network(n_inputs, n_layers, n_outputs)
train_network(network, train, learning_rate, n_epoch, n_outputs)

counter=0
for row in test:
    prediction = predict(network, row)
    actual=row[-1]
    if prediction==actual:
        counter+=1
print("The accuracy is " + str(counter/len(test)*100) + '%')


>epoch=0, lrate=0.100, error=160.653
>epoch=1, lrate=0.100, error=127.381
>epoch=2, lrate=0.100, error=125.300
>epoch=3, lrate=0.100, error=122.969
>epoch=4, lrate=0.100, error=120.013
>epoch=5, lrate=0.100, error=115.888
>epoch=6, lrate=0.100, error=109.844
>epoch=7, lrate=0.100, error=101.237
>epoch=8, lrate=0.100, error=90.289
>epoch=9, lrate=0.100, error=78.296
>epoch=10, lrate=0.100, error=66.746
>epoch=11, lrate=0.100, error=56.596
>epoch=12, lrate=0.100, error=48.170
>epoch=13, lrate=0.100, error=41.367
>epoch=14, lrate=0.100, error=35.907
>epoch=15, lrate=0.100, error=31.502
>epoch=16, lrate=0.100, error=27.907
>epoch=17, lrate=0.100, error=24.938
>epoch=18, lrate=0.100, error=22.456
>epoch=19, lrate=0.100, error=20.357
>epoch=20, lrate=0.100, error=18.563
>epoch=21, lrate=0.100, error=17.016
>epoch=22, lrate=0.100, error=15.671
>epoch=23, lrate=0.100, error=14.493
>epoch=24, lrate=0.100, error=13.456
>epoch=25, lrate=0.100, error=12.536
>epoch=26, lrate=0.100, error=11.718
>ep

>epoch=226, lrate=0.100, error=0.493
>epoch=227, lrate=0.100, error=0.490
>epoch=228, lrate=0.100, error=0.487
>epoch=229, lrate=0.100, error=0.485
>epoch=230, lrate=0.100, error=0.482
>epoch=231, lrate=0.100, error=0.479
>epoch=232, lrate=0.100, error=0.477
>epoch=233, lrate=0.100, error=0.474
>epoch=234, lrate=0.100, error=0.471
>epoch=235, lrate=0.100, error=0.469
>epoch=236, lrate=0.100, error=0.466
>epoch=237, lrate=0.100, error=0.464
>epoch=238, lrate=0.100, error=0.461
>epoch=239, lrate=0.100, error=0.459
>epoch=240, lrate=0.100, error=0.457
>epoch=241, lrate=0.100, error=0.454
>epoch=242, lrate=0.100, error=0.452
>epoch=243, lrate=0.100, error=0.449
>epoch=244, lrate=0.100, error=0.447
>epoch=245, lrate=0.100, error=0.445
>epoch=246, lrate=0.100, error=0.443
>epoch=247, lrate=0.100, error=0.440
>epoch=248, lrate=0.100, error=0.438
>epoch=249, lrate=0.100, error=0.436
>epoch=250, lrate=0.100, error=0.434
>epoch=251, lrate=0.100, error=0.431
>epoch=252, lrate=0.100, error=0.429
>

>epoch=448, lrate=0.100, error=0.211
>epoch=449, lrate=0.100, error=0.210
>epoch=450, lrate=0.100, error=0.210
>epoch=451, lrate=0.100, error=0.209
>epoch=452, lrate=0.100, error=0.209
>epoch=453, lrate=0.100, error=0.208
>epoch=454, lrate=0.100, error=0.207
>epoch=455, lrate=0.100, error=0.207
>epoch=456, lrate=0.100, error=0.206
>epoch=457, lrate=0.100, error=0.206
>epoch=458, lrate=0.100, error=0.205
>epoch=459, lrate=0.100, error=0.205
>epoch=460, lrate=0.100, error=0.204
>epoch=461, lrate=0.100, error=0.204
>epoch=462, lrate=0.100, error=0.203
>epoch=463, lrate=0.100, error=0.203
>epoch=464, lrate=0.100, error=0.202
>epoch=465, lrate=0.100, error=0.201
>epoch=466, lrate=0.100, error=0.201
>epoch=467, lrate=0.100, error=0.200
>epoch=468, lrate=0.100, error=0.200
>epoch=469, lrate=0.100, error=0.199
>epoch=470, lrate=0.100, error=0.199
>epoch=471, lrate=0.100, error=0.198
>epoch=472, lrate=0.100, error=0.198
>epoch=473, lrate=0.100, error=0.197
>epoch=474, lrate=0.100, error=0.197
>

In [18]:
import pickle
pickle.dump(network , open( 'text_network_weights.pkl' , 'wb' ) )